In [1]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\WilhelmB\Documents\DiagAssistAI


In [4]:
import os
os.environ['CUDA_HOME'] = 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.0'

In [2]:
!pip -V

pip 24.2 from C:\Users\WilhelmB\miniconda3\envs\prueba\lib\site-packages\pip (python 3.10)



In [1]:
%pip install -e .  --verbose

Using pip 24.0 from c:\Users\WilhelmB\miniconda3\envs\prueba\lib\site-packages\pip (python 3.10)
Obtaining file:///C:/Users/WilhelmB/Documents/DiagAssistAI
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for groundingdino (pyproject.toml): started
  Building editable for groundingdino (pyproject.toml): finished with status 'done'
  Created wheel for groundingdino: filename=groundingdino-0.0.0-0.editable-py3-none-any.whl size=5591 sha256=ddcc51b0c4c9fb6ba4e5b013d1e0d5f984bf0ffb2fefa6b01ecbfd121bb79017
  Stored in directory: C:\

  Running command pip subprocess to install build dependencies
    Using cached setuptools-72.1.0-py3-none-any.whl.metadata (6.6 kB)
    Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
    Using cached torch-2.4.0-cp310-cp310-win_amd64.whl.metadata (27 kB)
    Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
    Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
    Using cached sympy-1.13.2-py3-none-any.whl.metadata (12 kB)
    Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
    Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
    Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
    Using cached MarkupSafe-2.1.5-cp310-cp310-win_amd64.whl.metadata (3.1 kB)
    Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached setuptools-72.1.0-py3-none-any.whl (2.3 MB)
  Using cached wheel-0.44.0-py3-none-any.whl (67 kB)
  Using cached torch-2.4.0-cp310-cp310-win_amd64.whl (197.9 MB)
  Usi

In [14]:
import torch

In [3]:
import setuptools

In [4]:

%env CUDA_HOME=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6

env: CUDA_HOME=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6


In [2]:
%echo %CUDA_HOME% 

C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1


In [3]:
%cd C:\Users\WilhelmB\Documents\DiagAssistAI\src\groundingdino
from groundingdino import _C

C:\Users\WilhelmB\Documents\DiagAssistAI\src\groundingdino


c:\Users\WilhelmB\miniconda3\envs\DiagIA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\users\wilhelmb\documents\diagassistai\src\groundingdino\models\GroundingDINO\ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


ImportError: cannot import name '_C' from 'groundingdino' (c:\users\wilhelmb\documents\diagassistai\src\groundingdino\__init__.py)

In [3]:
%load_ext autoreload
%autoreload 2

In [6]:
import torch 
torch.version.cuda

'12.1'

In [3]:
!mkdir {HOME}\\build
%cd build
!cmake ..

[WinError 2] El sistema no puede encontrar el archivo especificado: 'build'
c:\Users\WilhelmB\Documents\DiagAssistAI\build


Ya existe el subdirectorio o el archivo c:\Users\WilhelmB\Documents\DiagAssistAI\\build.
C:\Users\WilhelmB\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


-- Building for: Visual Studio 17 2022
-- Selecting Windows SDK version 10.0.22621.0 to target Windows 10.0.22631.
-- The C compiler identification is MSVC 19.40.33813.0
-- The CXX compiler identification is MSVC 19.40.33813.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: C:/Program Files (x86)/Microsoft Visual Studio/2022/BuildTools/VC/Tools/MSVC/14.40.33807/bin/Hostx64/x64/cl.exe - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: C:/Program Files (x86)/Microsoft Visual Studio/2022/BuildTools/VC/Tools/MSVC/14.40.33807/bin/Hostx64/x64/cl.exe - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found CUDA: C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v12.1 (found version "12.1")
-- Configuring incomplete, errors occurred!


CMake Warning (dev) at CMakeLists.txt:7 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

CMake Error at CMakeLists.txt:8 (find_package):
  By not providing "FindTorch.cmake" in CMAKE_MODULE_PATH this project has
  asked CMake to find a package configuration file provided by "Torch", but
  CMake did not find one.

  Could not find a package configuration file provided by "Torch" with any of
  the following names:

    TorchConfig.cmake
    torch-config.cmake

  Add the installation prefix of "Torch" to CMAKE_PREFIX_PATH or set
  "Torch_DIR" to a directory containing one of the above files.  If "Torch"
  provides a separate development package or SDK, be sure it has been
  installed.




In [4]:
!pip list

Package                Version      Editable project location
---------------------- ------------ ----------------------------------------
addict                 2.4.0
asttokens              2.4.1
certifi                2024.7.4
charset-normalizer     3.3.2
colorama               0.4.6
comm                   0.2.2
contourpy              1.2.1
cycler                 0.12.1
debugpy                1.8.5
decorator              5.1.1
defusedxml             0.7.1
exceptiongroup         1.2.2
executing              2.0.1
filelock               3.13.1
fonttools              4.53.1
fsspec                 2024.2.0
groundingdino          0.0.0        C:\Users\WilhelmB\Documents\DiagAssistAI
huggingface-hub        0.23.5
idna                   3.7
importlib_metadata     8.2.0
ipykernel              6.29.5
ipython                8.26.0
jedi                   0.19.1
Jinja2                 3.1.3
jupyter_client         8.6.2
jupyter_core           5.7.2
kiwisolver             1.4.5
MarkupSafe         

In [4]:
from src.LangSAM.SamLangDino import SamDino
from src.DataSets.Mamitas_Thermal_Dataset.Mamitas_Dataset import Mamitas_Thermal_Feet_Dataset, ToBoolTensor, PermuteTensor 
from torchvision.transforms import transforms
import torch

c:\Users\WilhelmB\miniconda3\envs\prueba\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\WilhelmB\Documents\DiagAssistAI\src\groundingdino\models\GroundingDINO\ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


ModuleNotFoundError: No module named 'sklearn'

In [2]:
model = SamDino(SAM='vit_h')

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [4]:
transform_mask = transforms.Compose([
    ToBoolTensor(),
    PermuteTensor((1,2,0))
])
transform_img = transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.uint8),
    PermuteTensor((1,2,0))
])

_, val_dataset = Mamitas_Thermal_Feet_Dataset().generate_dataset_with_val(torch_dataset=True,
                                                   batch_size=38,
                                                   shuffle=False,
                                                   split_val=0.2,
                                                   merge_image=True,
                                                   transform_mask=transform_mask,
                                                   transform_img=transform_img)


Dataset URL: https://www.kaggle.com/datasets/lucasiturriago/mamitas-thermal-feet
Train_dataset: 150
Val_dataset: 38


In [5]:
text_prompt="feet,foot"
box_threshold=0.10
text_threshold=0.25
process_box_threshold = 0.10

boxes,logits,phrases,images,ids,masks_o,masks_p = model.LangSam_batch(image_data=val_dataset,
                                                                      text_prompt=text_prompt,
                                                                      box_threshold=box_threshold,
                                                                      process_box_threshold=process_box_threshold,
                                                                      text_threshold=text_threshold,
                                                                      mode_predict='box',
                                                                      mode_data='batch')

NameError: name '_C' is not defined